# 03 — Cosine Similarity (TF‑IDF & optional SBERT)

Ziel: Ähnlichkeit zwischen kurzen Mixing-Posts berechnen und Top‑Treffer anzeigen.

**Was du lernst**
- TF‑IDF Vektorisierung und Cosine Similarity
- Optional: Semantische Embeddings (SBERT) als Vergleich
- Einfache Query‑Funktionen und Mini‑Eval (sanity check)


## Setup & Daten laden
Wir laden den Korpus aus `data/sample_corpus.json`. Falls die Datei fehlt, nutzen wir einen kleinen Fallback.

In [ ]:
import json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel  # Cosine for L2-normalized vectors

DATA = Path("data"); DATA.mkdir(exist_ok=True)

def load_corpus():
    p = DATA/"sample_corpus.json"
    if p.exists():
        with p.open("r", encoding="utf-8") as f:
            x = json.load(f)
            if isinstance(x, list) and all(isinstance(t, str) for t in x):
                return x
    # Fallback (Woche 1 Demo)
    return [
        "Die Snare ist zu laut und harsch",
        "Kick zu weich, es fehlt der Punch",
        "Vocals klingen nasal, 800 Hz absenken",
        "Bass maskiert die Kick, Sidechain nötig",
        "S-Laute sind scharf, De-Esser einsetzen",
    ]

corpus = load_corpus()
len(corpus), corpus[:2]

## TF‑IDF aufbauen
Wir nutzen Unigramme und Bigramme. `linear_kernel` liefert bei normalisierten TF‑IDF‑Vektoren die **Cosine Similarity**.

In [ ]:
tfidf = TfidfVectorizer(lowercase=True, ngram_range=(1,2), min_df=1)
X = tfidf.fit_transform(corpus)
X.shape, len(tfidf.get_feature_names_out())

## Ähnlichkeit berechnen (TF‑IDF)
Die Funktion `rank_tfidf` gibt sortierte Dokumentindizes sowie die zugehörigen Similarity‑Scores zurück.

In [ ]:
def rank_tfidf(query: str, k: int = 5):
    qv = tfidf.transform([query])
    sims = linear_kernel(qv, X).ravel()  # entspricht Cosine, da TF‑IDF normiert
    order = np.argsort(-sims)
    topk = order[:k]
    return topk.tolist(), sims[topk].tolist()

# Sanity‑Check
idxs, scores = rank_tfidf("snare zu laut", k=min(5, len(corpus)))
pd.DataFrame({"rank": np.arange(1, len(idxs)+1), "doc": [corpus[i] for i in idxs], "score": np.round(scores, 3)})

## Optional: SBERT (semantisch, CPU)
Falls `sentence_transformers` und `torch` verfügbar sind, vergleichen wir die Resultate. Bei Triton‑Konflikten (macOS/CPU) deinstalliere `triton` und pinne Torch (z. B. `torch==2.2.2`).

In [ ]:
def try_sbert(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    try:
        from sentence_transformers import SentenceTransformer
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model = SentenceTransformer(model_name, device="cpu")
        doc_emb = model.encode(corpus, normalize_embeddings=True)
        def rank(query: str, k: int = 5):
            qv = model.encode([query], normalize_embeddings=True)
            sims = (qv @ doc_emb.T).ravel()
            order = np.argsort(-sims)
            topk = order[:k]
            return topk.tolist(), sims[topk].tolist()
        return rank
    except Exception as e:
        print("SBERT nicht verfügbar:", e)
        return None

rank_sbert = try_sbert()
rank_sbert

## Vergleich: TF‑IDF vs. SBERT (für eine Query)
Wir vergleichen die Top‑Treffer der beiden Methoden.


In [ ]:
query = "kick zu weich"
k = min(5, len(corpus))

tf_idx, tf_sc = rank_tfidf(query, k)
tf_df = pd.DataFrame({
    "rank": np.arange(1, len(tf_idx)+1),
    "method": "tfidf",
    "doc": [corpus[i] for i in tf_idx],
    "score": np.round(tf_sc, 3)
})

if rank_sbert:
    sb_idx, sb_sc = rank_sbert(query, k)
    sb_df = pd.DataFrame({
        "rank": np.arange(1, len(sb_idx)+1),
        "method": "sbert",
        "doc": [corpus[i] for i in sb_idx],
        "score": np.round(sb_sc, 3)
    })
    out = pd.concat([tf_df, sb_df], ignore_index=True)
else:
    out = tf_df

out

## Mini‑Eval (sanity check)
Kleines Ground‑Truth Mapping: Query → relevante Dokument‑Indizes. Wir berechnen **MRR** und **Precision@k** (k=3) für TF‑IDF (und SBERT, falls verfügbar).

In [ ]:
GT = {
    "snare zu laut": [0],
    "kick zu weich": [1],
    "vocals nasal 800 hz": [2],
    "bass maskiert kick sidechain": [3],
    "s-laute scharf de-esser": [4],
}

def precision_at_k(relevants, retrieved, k=3):
    R = set(relevants); topk = retrieved[:k]
    hits = sum(1 for i in topk if i in R)
    return hits / max(1, len(topk))

def reciprocal_rank(relevants, retrieved):
    R = set(relevants)
    for r, idx in enumerate(retrieved, 1):
        if idx in R:
            return 1.0 / r
    return 0.0

def evaluate(run_name, rank_fn):
    rows = []
    for q, rel in GT.items():
        idxs, _ = rank_fn(q, k=min(5, len(corpus)))
        rows.append({
            "run": run_name,
            "query": q,
            "MRR": reciprocal_rank(rel, idxs),
            "P@3": precision_at_k(rel, idxs, 3)
        })
    df = pd.DataFrame(rows)
    agg = df.drop(columns=["query"]).groupby("run").mean(numeric_only=True).round(3)
    return df, agg

tf_df, tf_agg = evaluate("tfidf", rank_tfidf)
display(tf_agg); display(tf_df)

if rank_sbert:
    sb_df, sb_agg = evaluate("sbert", rank_sbert)
    display(sb_agg); display(sb_df)

## Übungen (kurz)
1. Erweitere den Korpus (10–20 Zeilen) und beobachte, wie sich die Rankings ändern.
2. Ändere die `ngram_range` auf `(1,3)` und vergleiche.
3. Füge einfache Synonyme/Query‑Expansion hinzu (z. B. *kick* → *bassdrum*), bevor du `rank_tfidf` aufrufst.
4. (Optional) Evaluiere verschiedene TF‑IDF‑Parameter (z. B. Stopwörter, `min_df`).
